In [2]:
import pandas as pd
import gender_guesser.detector as gender
from genderize import Genderize
import requests
import matplotlib.pyplot as plt
from scipy.stats import pointbiserialr
import numpy as np
import seaborn as sns

In [4]:
pelis2 = pelis = pd.read_csv(r"C:\Users\Genís\Desktop\IRONHACK\Módulo 2\Projecte/pelis.csv")
pelis2

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,gender,ISO_codes,gender_num
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,1.900000e+07,46998772.0,Warner Bros.,146.0,male,GB,0
1,The Blue Lagoon,R,Adventure,1980,"July 2, 1980 (United States)",5.8,65000.0,Randal Kleiser,Henry De Vere Stacpoole,Brooke Shields,United States,4.500000e+06,58853106.0,Columbia Pictures,104.0,male,US,0
2,Star Wars: Episode V - The Empire Strikes Back,PG,Action,1980,"June 20, 1980 (United States)",8.7,1200000.0,Irvin Kershner,Leigh Brackett,Mark Hamill,United States,1.800000e+07,538375067.0,Lucasfilm,124.0,male,US,0
3,Airplane!,PG,Comedy,1980,"July 2, 1980 (United States)",7.7,221000.0,Jim Abrahams,Jim Abrahams,Robert Hays,United States,3.500000e+06,83453539.0,Paramount Pictures,88.0,male,US,0
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6.000000e+06,39846344.0,Orion Pictures,98.0,male,US,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7663,More to Life,NaN,Drama,2020,"October 23, 2020 (United States)",3.1,18.0,Joseph Ebanks,Joseph Ebanks,Shannon Bond,United States,7.000000e+03,NaN,NaN,90.0,male,US,0
7664,Dream Round,NaN,Comedy,2020,"February 7, 2020 (United States)",4.7,36.0,Dusty Dukatz,Lisa Huston,Michael Saquella,United States,3.558988e+07,NaN,Cactus Blue Entertainment,90.0,male,US,0
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,5.875000e+04,NaN,Embi Productions,NaN,male,US,0
7666,It's Just Us,NaN,Drama,2020,"October 1, 2020 (United States)",NaN,NaN,James Randall,James Randall,Christina Roz,United States,1.500000e+04,NaN,NaN,120.0,male,US,0


Buscamos géneros de los actores usando gender_guesser y la api Genderize:

In [5]:
pelis2['star'].isnull().sum()

1

In [16]:
d = gender.Detector()

In [18]:
pelis2['gender_star'] = pelis2['star'].apply(lambda x: d.get_gender(x.split()[0]) if isinstance(x, str) else '')

Pasamos los mostly male o female:

In [21]:
pelis2['gender_star'] = pelis2['gender_star'].replace({'mostly_male': 'male', 'mostly_female': 'female', 'andy': 'male'})

In [22]:
pelis2['gender_star'].value_counts()

male       5283
female     1908
unknown     476
              1
Name: gender_star, dtype: int64

In [24]:
pelis2 = pelis2.rename(columns={'gender': 'gender_director'})

De nuevo tenemos que limpiar los unknown:

In [47]:
import requests

def detect_gender(name, country_code):
    try:
        response = requests.get(f"https://api.genderize.io/?name={name}&country_id={country_code}")
        data = response.json()
        gender = data['gender']
    except (KeyError, ValueError):
        gender = "male"
    return gender

for i, row in pelis2.iterrows():
    if row['gender_star'] == 'Unknown':
        # llamar a la API Genderize.io para obtener el género del director
        first_name = row['star'].split()[0]  # Tomar el primer nombre
        gender = detect_gender(first_name, row['ISO_codes'])
        if gender:
            pelis2.at[i, 'gender_star'] = gender
        else:
            pelis2.at[i, 'gender_star'] = 'male'


In [48]:
pelis2['gender_star'].value_counts()

male      5759
female    1908
             1
Name: gender_star, dtype: int64

In [46]:
pelis2[pelis2['gender_star'] == 'Unknown']

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime,gender_director,ISO_codes,gender_num,gender_star,gender
4,Caddyshack,R,Comedy,1980,"July 25, 1980 (United States)",7.3,108000.0,Harold Ramis,Brian Doyle-Murray,Chevy Chase,United States,6.000000e+06,39846344.0,Orion Pictures,98.0,male,US,0,Unknown,Unknown
11,The Gods Must Be Crazy,PG,Adventure,1980,"October 26, 1984 (United States)",7.3,54000.0,Jamie Uys,Jamie Uys,N!xau,South Africa,5.000000e+06,30031783.0,C.A.T. Films,109.0,female,ZA,1,Unknown,Unknown
44,The Awakening,R,Horror,1980,"October 31, 1980 (United States)",4.8,2000.0,Mike Newell,Allan Scott,Charlton Heston,United Kingdom,3.558988e+07,8415112.0,EMI Films,101.0,male,GB,0,Unknown,Unknown
45,Cheech and Chong's Next Movie,R,Comedy,1980,"July 18, 1980 (United States)",6.1,12000.0,Tommy Chong,Tommy Chong,Cheech Marin,United States,3.558988e+07,41675194.0,Universal Pictures,99.0,male,US,0,Unknown,Unknown
57,Oh Heavenly Dog,PG,Comedy,1980,"July 11, 1980 (United States)",5.4,2400.0,Joe Camp,Rod Browning,Chevy Chase,United States,6.000000e+06,6216067.0,Mulberry Square Productions,103.0,male,US,0,Unknown,Unknown
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7652,The Eight Hundred,Not Rated,Action,2020,"August 28, 2020 (United States)",6.8,3700.0,Hu Guan,Hu Guan,Zhi-zhong Huang,China,8.000000e+07,461421559.0,Beijing Diqi Yinxiang Entertainment,149.0,male,CN,0,Unknown,Unknown
7655,Legend of Deification,TV-PG,Animation,2020,"October 1, 2020 (United States)",6.6,1300.0,Teng Cheng,NaN,Guangtao Jiang,China,3.558988e+07,240663149.0,Beijing Enlight Pictures,110.0,male,CN,0,Unknown,Unknown
7660,Aloha Surf Hotel,NaN,Comedy,2020,"November 5, 2020 (United States)",7.1,14.0,Stefan C. Schaefer,Stefan C. Schaefer,Augie Tulba,United States,3.558988e+07,NaN,Abominable Pictures,90.0,male,US,0,Unknown,Unknown
7665,Saving Mbango,NaN,Drama,2020,"April 27, 2020 (Cameroon)",5.7,29.0,Nkanya Nkwai,Lynno Lovert,Onyama Laura,United States,5.875000e+04,NaN,Embi Productions,NaN,male,US,0,Unknown,Unknown


Una vez todo limpio guardamos el csv. Pero antes borramos las columnas sobrantes:

In [54]:
pelis2 = pelis2.drop('gender', axis=1)
pelis2 = pelis2.drop('gender_num', axis=1)

In [56]:
pelis2 = pelis2[['name', 'rating', 'genre', 'year', 'released', 'score', 'votes', 'director', 'gender_director', 'writer', 'star', 'gender_star', 'country', 'ISO_codes', 'budget', 'gross', 'company', 'runtime']]

In [58]:
pelis2.to_csv('pelis2.csv', index=False)